As noted in Tutorial 00, `pygsti` can provide many reports or presentation slides.

In this tutorial, we look at how to customize some of the output of `pygsti`'s report generating module.

In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

First, we'll make smaller reports that don't contain any confidence intervals for the estimates,
nor any appendices.

NOTE: In order for you to create PowerPoint files, you'll have to have `python-pptx` installed on your system.

In [2]:
# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
gs_target.set_basis("gm",2)
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of MC2GST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

go_gatesets = [ pygsti.optimize_gauge(gs, "target", targetGateset=gs_target, gateWeight=1, spamWeight=0.001) for gs in gs_lsgst_list]

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache
--- LGST ---
    
  --- Iterative MC2GST: Iter 0 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 40.9238 (92 data params - 40 model params = expected mean of 52; p-value = 0.866034)
    
  --- Iterative MC2GST: Iter 1 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 40.9238 (92 data params - 40 model params = expected mean of 52; p-value = 0.866034)
    
  --- Iterative MC2GST: Iter 2 of 9  168 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 124.56 (168 data params - 40 model params = expected mean of 128; p-value = 0.569533)
    
  --- Iterative MC2GST: Iter 3 of 9  441 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 424.144 (441 data params - 40 model params = expected mean of 401; p-value = 0.204559)
    
  --- Iterative MC2GST: Iter 4 of 9  817 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 772.131 (817 data params - 40

In [3]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    go_gatesets, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False)

In [4]:
#Make smaller reports (no confidence intervals or appendicies)
res.create_full_report_pdf(filename="tutorial_files/Example_reportB.pdf", verbosity=2, confidenceLevel=None,
                        debugAidsAppendix=False, gaugeOptAppendix=False,
                        pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_brief_report_pdf(filename="tutorial_files/Example_briefB.pdf", verbosity=2, confidenceLevel=None)

res.create_presentation_pdf(filename="tutorial_files/Example_slidesB.pdf", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_presentation_ppt(filename="tutorial_files/Example_slidesB.ppt", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

  *** Generating tables ***
   Iter 00 of 16 :   Generating table: targetSpamTable
   Iter 01 of 16 :   Generating table: targetGatesTable
   Iter 02 of 16 :   Generating table: datasetOverviewTable
   Iter 03 of 16 :   Generating table: bestGatesetSpamTable
   Iter 04 of 16 :   Generating table: bestGatesetSpamParametersTable
   Iter 05 of 16 :   Generating table: bestGatesetGatesTable
   Iter 06 of 16 :   Generating table: bestGatesetChoiTable
   Iter 07 of 16 :   Generating table: bestGatesetDecompTable
   Iter 08 of 16 :   Generating table: bestGatesetRotnAxisTable
   Iter 09 of 16 :   Generating table: bestGatesetClosestUnitaryTable


/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:233: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m = zeros((N, M), dtype=dtype)
/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m[:M-k].flat[i::M+1] = 1


   Iter 10 of 16 :   Generating table: bestGatesetVsTargetTable
   Iter 11 of 16 :   Generating table: bestGatesetErrorGenTable
   Iter 12 of 16 :   Generating table: fiducialListTable
   Iter 13 of 16 :   Generating table: prepStrListTable
   Iter 14 of 16 :   Generating table: effectStrListTable
   Iter 15 of 16 :   Generating table: germListTable
   Iter 16 of 16 :   Generating table: progressTable
  *** Generating plots ***
  Chi2 plots (2): 
   Iter 0 of 1 :   Generating figure: bestEstimateColorBoxPlot
   Iter 1 of 1 :   Generating figure: invertedBestEstimateColorBoxPlot
  
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/Example_reportB.pdf successfully generated.
  Final output PDF tutorial_files/Example_reportB.pdf successfully generated. Cleaning up .aux and .log files.
  *** Generating tables ***
  Retrieving cached table: bestGatesetSpamTable
  Retrieving cached table: b

In [5]:
res.create_full_report_pdf(filename="tutorial_files/Example_report.pdf", verbosity=2, confidenceLevel=95,
                        debugAidsAppendix=True, gaugeOptAppendix=True,
                        pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_brief_report_pdf(filename="tutorial_files/Example_brief.pdf", verbosity=2, confidenceLevel=95)

res.create_presentation_pdf(filename="tutorial_files/Example_slides.pdf", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_presentation_ppt(filename="tutorial_files/Example_slides.ppt", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

      
  --- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
  205s           0.0183866343
  259s           0.0182468335
  289s           0.0181447444
  319s           0.0180700722
  350s           0.0180189904
  381s           0.0179935924
  412s           0.0179572650
  443s           0.0179483455
  475s           0.0179394972
  506s           0.0179351061
  538s           0.0179322957
  569s           0.0179295563
  600s           0.0179276138
  631s           0.0179260339
  662s           0.0179252069
    The resulting min sqrt(sum(gateCIs**2)): 0.0179252
  *** Generating tables ***
   Iter 00 of 16 :   Generating table: targetSpamTable (w/95% CIs)
   Iter 01 of 16 :   Generating table: targetGatesTable (w/95% CIs)
   Iter 02 of 16 :   Generating table: datasetOverviewTable (w/95% CIs)
   Iter 03 of 16 :   Generating table: bestGatesetSpamTable (w/95% CIs)
   Iter 04 of 16 :   Generating table: bestGatesetSpamParametersTable (w/95% CIs)
   Iter 05 of 16 :   Generating ta

/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:233: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m = zeros((N, M), dtype=dtype)
/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m[:M-k].flat[i::M+1] = 1


   Iter 10 of 16 :   Generating table: bestGatesetVsTargetTable (w/95% CIs)
   Iter 11 of 16 :   Generating table: bestGatesetErrorGenTable (w/95% CIs)
   Iter 12 of 16 :   Generating table: fiducialListTable (w/95% CIs)
   Iter 13 of 16 :   Generating table: prepStrListTable (w/95% CIs)
   Iter 14 of 16 :   Generating table: effectStrListTable (w/95% CIs)
   Iter 15 of 16 :   Generating table: germListTable (w/95% CIs)
   Iter 16 of 16 :   Generating table: progressTable (w/95% CIs)
  Generating special: gaugeOptAppendixTables (w/95% CIs)
  Generating special: gaugeOptAppendixTables
  Generating special: gaugeOptAppendixGatesets (w/95% CIs)
  Generating special: gaugeOptAppendixGatesets
  Performing gauge transforms for appendix...
--- Gauge Optimization to a target (L-BFGS-B) ---
  The resulting Frobenius-norm distance is: 0.0387358
    frobenius norm diff of Gi = 0.172439
    frobenius norm diff of Gx = 0.172062
    frobenius norm diff of Gy = 0.175132
    frobenius norm diff of rho

/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:2758: RuntimeWarning: invalid value encountered in absolute
  for iy in range(len(yvals)) ])
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:413: RuntimeWarning: invalid value encountered in greater
  resdat[resdat>0] /= abs(vmax - midpoint)
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:414: RuntimeWarning: invalid value encountered in less
  resdat[resdat<0] /= abs(vmin - midpoint)


  
   -- Error rate plots...
  Generating figure: smallEigvalErrRateColorBoxPlot (w/95% CIs)
  Generating figure: smallEigvalErrRateColorBoxPlot
  Retrieving cached special: DirectLongSeqGatesets (w/95% CIs)


...

   -- Whack-a-mole plots (6):    Iter 0 of 2 :   Generating figure: whackGxMoleBoxes (w/95% CIs)
  Generating figure: whackGxMoleBoxes
   Iter 1 of 2 :   Generating figure: whackGyMoleBoxes (w/95% CIs)
  Generating figure: whackGyMoleBoxes
   Iter 2 of 2 :   Generating figure: whackGiMoleBoxes (w/95% CIs)
  Generating figure: whackGiMoleBoxes
   Iter 0 of 2 :   Generating figure: whackGxMoleBoxesSummed (w/95% CIs)
  Generating figure: whackGxMoleBoxesSummed
   Iter 1 of 2 :   Generating figure: whackGyMoleBoxesSummed (w/95% CIs)
  Generating figure: whackGyMoleBoxesSummed
   Iter 2 of 2 :   Generating figure: whackGiMoleBoxesSummed (w/95% CIs)
  Generating figure: whackGiMoleBoxesSummed
  
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/Example_report.pdf successfully generated.
  Final output PDF tutorial_files/Example_report.pdf successfully generated. Cleaning up .aux and .log fi

/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:413: RuntimeWarning: invalid value encountered in greater
  resdat[resdat>0] /= abs(vmax - midpoint)
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:414: RuntimeWarning: invalid value encountered in less
  resdat[resdat<0] /= abs(vmin - midpoint)


  
   -- Error rate plots...
  Retrieving cached figure: smallEigvalErrRateColorBoxPlot (w/95% CIs)
   -- Whack-a-mole plots (6):    Iter 0 of 2 :   Retrieving cached figure: whackGxMoleBoxes (w/95% CIs)
   Iter 1 of 2 :   Retrieving cached figure: whackGyMoleBoxes (w/95% CIs)
   Iter 2 of 2 :   Retrieving cached figure: whackGiMoleBoxes (w/95% CIs)
   Iter 0 of 2 :   Retrieving cached figure: whackGxMoleBoxesSummed (w/95% CIs)
   Iter 1 of 2 :   Retrieving cached figure: whackGyMoleBoxesSummed (w/95% CIs)
   Iter 2 of 2 :   Retrieving cached figure: whackGiMoleBoxesSummed (w/95% CIs)
  
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/Example_slides.pdf successfully generated.
  Final output PDF tutorial_files/Example_slides.pdf successfully generated. Cleaning up .aux and .log files.
  *** Generating tables ***
   Iter 00 of 15 :   Retrieving cached table: targetSpamTable (w/95% CI

/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:413: RuntimeWarning: invalid value encountered in greater
  resdat[resdat>0] /= abs(vmax - midpoint)
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:414: RuntimeWarning: invalid value encountered in less
  resdat[resdat<0] /= abs(vmin - midpoint)


  
   -- Error rate plots...
  Retrieving cached figure: smallEigvalErrRateColorBoxPlot (w/95% CIs)
   -- Whack-a-mole plots (6):    Iter 0 of 2 :   Retrieving cached figure: whackGxMoleBoxes (w/95% CIs)
   Iter 1 of 2 :   Retrieving cached figure: whackGyMoleBoxes (w/95% CIs)
   Iter 2 of 2 :   Retrieving cached figure: whackGiMoleBoxes (w/95% CIs)
   Iter 0 of 2 :   Retrieving cached figure: whackGxMoleBoxesSummed (w/95% CIs)
   Iter 1 of 2 :   Retrieving cached figure: whackGyMoleBoxesSummed (w/95% CIs)
   Iter 2 of 2 :   Retrieving cached figure: whackGiMoleBoxesSummed (w/95% CIs)
  
  *** Assembling PPT file ***
  Latexing progressTable table...
  Latexing bestGatesetVsTargetTable table...
  Latexing bestGatesetErrorGenTable table...
  Latexing bestGatesetDecompTable table...
  Latexing bestGatesetRotnAxisTable table...
  Latexing bestGatesetGatesTable table...
  Latexing bestGatesetSpamTable table...
  Latexing bestGatesetSpamParametersTable table...
  Latexing bestGatesetChoiTab

In [6]:
import pygsti.report.latex as LU
from imp import reload
reload(LU)
LU.latex_value(1.2e30,2)

'1\\e{30}'